# VacationPy
----

In [80]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint

# Import API key
from config import gkey

In [2]:
# configuring gmaps with unique key
gmaps.configure(api_key=gkey)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
csv_filepath = "../output_data/city_temps.csv"

cities_temp_df = pd.read_csv(csv_filepath, index_col=0)
cities_temp_df

,City,Country,Latitude,Longitude,Temperature,Humidity,Wind Speed,Cloudiness %
0,Mar del Plata,AR,-38.0023,-57.5575,63.16,77,12.66,75
1,Tuktoyaktuk,CA,69.4541,-133.0374,-23.80,76,2.35,75
2,Verkh-Usugli,RU,52.7000,115.1667,-23.51,82,3.31,74
3,Hilo,US,19.7297,-155.0900,73.58,61,10.36,20
4,Klaksvík,FO,62.2266,-6.5890,21.20,93,2.30,96
...,...,...,...,...,...,...,...,...
587,Ukiah,US,39.1502,-123.2078,53.60,62,6.91,90
588,Shizunai-furukawachō,JP,42.3339,142.3669,29.71,90,7.36,20
589,Flinders,AU,-34.5833,150.8552,78.31,72,4.00,67
590,Hammerfest,NO,70.6634,23.6821,26.60,86,11.50,75


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# creating sets of lats and longs to read into gmaps
locations = cities_temp_df[["Latitude", "Longitude"]]

# creating weights for heatmap - replacing any na values with 0
weights = cities_temp_df["Humidity"]
weights.fillna(0.0)

0      77
1      76
2      82
3      61
4      93
       ..
587    62
588    90
589    72
590    86
591    64
Name: Humidity, Length: 592, dtype: int64

In [46]:
# creating figure - centered at lat,lng = 0,0
fig = gmaps.figure(zoom_level=2, center=(0,0))

# Create heat layer 
heat_layer = gmaps.heatmap_layer(
    locations, 
    weights=weights, 
    dissipating=False, 
    max_intensity=21,
    point_radius=2
)

# Adding heatmap layer
fig.add_layer(heat_layer)

# Displaying figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

Conditions: 
- Between 50-70 degrees
- Cloudiness between 5-25%
- Humidity between 5-45%

In [22]:
cities_temp_df.columns

Index(['City', 'Country', 'Latitude', 'Longitude', 'Temperature', 'Humidity',
       'Wind Speed', 'Cloudiness %'],
      dtype='object')

In [37]:
# setting weather conditions
is_right_temp = (cities_temp_df["Temperature"] >= 50.0) & (cities_temp_df["Temperature"] <= 80.0)
is_cloudy = (cities_temp_df["Cloudiness %"] >= 5.0) & (cities_temp_df["Cloudiness %"] <= 35.0)
is_humid = (cities_temp_df["Humidity"] >= 5.0) & (cities_temp_df["Humidity"] <= 55.0)
is_perfect = (is_right_temp) & (is_cloudy) & (is_humid)

In [38]:
perfect_weather_df = cities_temp_df.loc[
    is_perfect,
    :
]
perfect_weather_df

,City,Country,Latitude,Longitude,Temperature,Humidity,Wind Speed,Cloudiness %
98,San Patricio,US,28.0170,-97.5169,67.71,26,10.36,20
207,Mataura,NZ,-46.1927,168.8643,74.41,41,4.56,30
233,Rawson,AR,-43.3002,-65.1023,69.30,49,26.55,13
234,Mumias,KE,0.3351,34.4864,62.49,43,7.90,34
236,San Miguel,SV,13.4833,-88.1833,78.67,48,3.04,14
265,Raja,SS,8.4596,25.6780,65.98,26,2.91,10
302,Comodoro Rivadavia,AR,-45.8667,-67.5000,69.80,52,12.66,20
346,Tuatapere,NZ,-46.1333,167.6833,70.41,52,4.94,25
351,Te Anau,NZ,-45.4167,167.7167,70.36,51,1.90,15
451,Corpus Christi,US,27.8006,-97.3964,67.60,32,11.50,20


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [54]:
perfect_weather_df.insert(8, "Hotel Name", "")

ValueError: cannot insert Hotel Name, already exists

In [57]:
hotel_df = perfect_weather_df
hotel_df

,City,Country,Latitude,Longitude,Temperature,Humidity,Wind Speed,Cloudiness %,Hotel Name
98,San Patricio,US,28.0170,-97.5169,67.71,26,10.36,20,
207,Mataura,NZ,-46.1927,168.8643,74.41,41,4.56,30,
233,Rawson,AR,-43.3002,-65.1023,69.30,49,26.55,13,
234,Mumias,KE,0.3351,34.4864,62.49,43,7.90,34,
236,San Miguel,SV,13.4833,-88.1833,78.67,48,3.04,14,
265,Raja,SS,8.4596,25.6780,65.98,26,2.91,10,
302,Comodoro Rivadavia,AR,-45.8667,-67.5000,69.80,52,12.66,20,
346,Tuatapere,NZ,-46.1333,167.6833,70.41,52,4.94,25,
351,Te Anau,NZ,-45.4167,167.7167,70.36,51,1.90,15,
451,Corpus Christi,US,27.8006,-97.3964,67.60,32,11.50,20,


In [110]:
perfect_coordinates = hotel_df[["Latitude", "Longitude"]]
perfect_coordinates = perfect_coordinates.reset_index(drop=True)

perfect_coordinates

,Latitude,Longitude
0,28.0170,-97.5169
1,-46.1927,168.8643
2,-43.3002,-65.1023
3,0.3351,34.4864
4,13.4833,-88.1833
5,8.4596,25.6780
6,-45.8667,-67.5000
7,-46.1333,167.6833
8,-45.4167,167.7167
9,27.8006,-97.3964


In [145]:
# setting search parameters
target_search = "hotel"
target_radius = 5000

# creating empty list for hotel name
hotel_name = []

# looping over perfect coordinat list 
for index, row in perfect_coordinates.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    params = {
        "location":f'{lat}, {lng}',
        "keyword":target_search,
        "radius":target_radius,
        "key":gkey
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

    response = requests.get(base_url, params=params).json()
    
    try:
        index_num.append(index)
        hotel_name.append(response["results"][0]["name"])
    except:
        hotel_name.append(" ") #adding null value to list
        print(f"Cannot find a hotel in city located at {lat}, {lng} indexed at {index}")

Cannot find a hotel in city located at 8.4596, 25.678 indexed at 5
Cannot find a hotel in city located at 12.2833, -10.9667 indexed at 10


In [146]:
hotel_df["Hotel Name"] = hotel_name

<ipython-input-146-a071b8cf7ee5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = hotel_name


In [147]:
hotel_df

,City,Country,Latitude,Longitude,Temperature,Humidity,Wind Speed,Cloudiness %,Hotel Name
98,San Patricio,US,28.0170,-97.5169,67.71,26,10.36,20,Americas Best Value Inn Sinton
207,Mataura,NZ,-46.1927,168.8643,74.41,41,4.56,30,Southland Organic Farmstay
233,Rawson,AR,-43.3002,-65.1023,69.30,49,26.55,13,Casa de mar
234,Mumias,KE,0.3351,34.4864,62.49,43,7.90,34,Number Seventy Guest House
236,San Miguel,SV,13.4833,-88.1833,78.67,48,3.04,14,Hotel Comfort Inn San Miguel
265,Raja,SS,8.4596,25.6780,65.98,26,2.91,10,
302,Comodoro Rivadavia,AR,-45.8667,-67.5000,69.80,52,12.66,20,Lucania Palazzo Hotel
346,Tuatapere,NZ,-46.1333,167.6833,70.41,52,4.94,25,Tui Base Camp
351,Te Anau,NZ,-45.4167,167.7167,70.36,51,1.90,15,Lakefront Lodge Te Anau
451,Corpus Christi,US,27.8006,-97.3964,67.60,32,11.50,20,Emerald Beach Hotel


In [151]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [152]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, hover_text=hotel_info)

# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))